In [1]:
import os
import numpy as np

import fitsio
from astropy.table import Table
import matplotlib.pyplot as plt

In [2]:
import legacyhalos.io
from legacyhalos.misc import lambda2mhalo, statsinbins, legacyhalos_plot_style

In [3]:
sns = legacyhalos_plot_style()
%matplotlib inline

In [ ]:
lsphot = legacyhalos.io.read_results(extname='LHPHOT', verbose=True)
lsphot[:2]

In [ ]:
parent = legacyhalos.io.read_parent(extname='LSPHOT', upenn=True, verbose=True)
parent[:2]

In [ ]:
def getmag(flux):
    mag = 22.5 - 2.5 * np.log10( flux )
    return mag

In [ ]:
lsmag = getmag(parent['flux_r'])
lhmag = getmag(lsphot['flux_r'])
good = np.isfinite(lsmag) * np.isfinite(lhmag)
lsmag = lsmag[good]
lhmag = lhmag[good]
dmag = lsmag - lhmag
stats = statsinbins(lhmag, dmag, binsize=0.3, minpts=10)
stats['xbin']

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
#ax.hexbin(lsmag, dmag, gridsize=100, mincnt=1, bins='log', 
#          cmap='viridis', extent=(15, 18.5, -0.5, 0.5))
ax.scatter(lhmag, dmag, marker='s', edgecolor='k', alpha=0.75, s=10)
ax.plot(stats['xbin'], stats['median'], lw=2, ls='-', color='k')
ax.plot(stats['xbin'], stats['q75'], lw=2, ls='--', color='k')
ax.plot(stats['xbin'], stats['q25'], lw=2, ls='--', color='k')

ax.set_xlim(13.5, 18.5)
ax.set_ylim(-0.5, 0.5)
ax.axhline(y=0, ls='--', color='gray')
ax.set_xlabel(r'$r_{Total}$ (AB mag)')
ax.set_ylabel(r'$r_{LegacySurvey}\ -\ r_{Total}$')

In [ ]:
stop

In [ ]:
ls = Table(fits.getdata(parentfile, extname='LSPHOT-ISEDFIT'))
ls

In [ ]:
mhalo = np.log10(lambda2mhalo(cat.lambda_chisq, redshift=cat.z))
mhalo

In [ ]:
mstar_med, bin_edges, _ = binned_statistic(mhalo, cat.mstar_avg, statistic='median', bins=bins)
bin_width = (bin_edges[1] - bin_edges[0])
mhalo_med = bin_edges[1:] - bin_width/2
print(bin_width)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

colors = iter(sns.color_palette())

rich = cat.lambda_chisq > 100
ax.plot(mhalo_med, mstar_med, color='k', ls='-', lw=3, alpha=0.5)
ax.plot(mhalo_med, mstar_p75, color='k', ls='--', lw=3, alpha=0.5)
ax.plot(mhalo_med, mstar_p25, color='k', ls='--', lw=3, alpha=0.5)

g = ax.errorbar(gonz['m500'], gonz['mbcg'], yerr=gonz['mbcg_err'], color=next(colors),
                fmt='o', label='Gonzalez+13', markersize=10)
k = ax.errorbar(krav['m500'], krav['mbcg'], yerr=krav['mbcg_err'], color=next(colors),
                fmt='s', label='Kravtsov+14', markersize=10)
r = ax.scatter(mhalo[rich], cat.mstar_avg[rich], alpha=0.9, color=next(colors),
               edgecolor='k', marker='D', s=50, label=r'redMaPPer ($\lambda>100$)')

ax.text(0.12, 0.16, 'redMaPPer\n$0.1<z<0.3$', multialignment='center',
        transform=ax.transAxes, fontsize=14)
m500 = np.linspace(13.55, 15.25, 50)
ff = ax.plot(m500, np.polyval([0.33, 12.24], m500-14.5), ls='-', 
            color='k', label=r'$M_{*}\propto M_{500}^{0.33}$')
ax.text(0.12, 0.9, r'$M_{*}\propto M_{500}^{0.33}$', multialignment='center',
        transform=ax.transAxes, fontsize=16)
ax.plot([13.55, 13.68], [12.8, 12.8], ls='-', color='k') # hack!!!

#ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.xaxis.set_major_locator(MultipleLocator(0.5))

hh = [g, k, r]
ax.legend(hh, [H.get_label() for H in hh], loc='lower right',
          frameon=True, fontsize=16)
#ax.legend(ff, ff.get_label(), loc='upper left',
#          frameon=True, fontsize=16)
#ax.legend(loc='upper left', frameon=True, fontsize=16)

ax.set_ylim(10.5, 13)
ax.set_xlim(13.5, 15.3)
ax.set_xlabel(r'$\log_{10}\, (M_{500}\ /\ M_{\odot})$')
ax.set_ylabel(r'$\log_{10}\, (M_{*}\ /\ M_{\odot})$')

In [ ]:
sns.jointplot(ls['MSTAR_AVG'], sdss['MSTAR_AVG'] - ls['MSTAR_AVG'], 
              kind="kde", color="#4CB391", xlim=(10, 13), ylim=(-0.5, 0.5))
sns.kdeplot(ls['MSTAR_AVG'], sdss['MSTAR_AVG'] - ls['MSTAR_AVG'], 
            cmap="Blues", shade=True, shade_lowest=True, cbar=True,
            cut=0,
           )